In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import to_categorical
from subprocess import check_output
print(check_output(['ls', 'input']).decode('utf8'))

Using TensorFlow backend.


class_maps.json
sample_submission.csv
test_
test.csv
train_
train.csv



In [2]:
train_img_path = 'input/train_/'
test_img_path = 'input/test_/'
IM_WIDTH, IM_HEIGHT = 139, 139 #fixed size for InceptionV3
NB_EPOCHS = 10
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 120
le = LabelEncoder()

In [3]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [4]:
def get_img_cv2(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    resized = cv2.resize(img,(IM_WIDTH, IM_HEIGHT), cv2.INTER_LINEAR)
    return resized

In [5]:
def load_train():
    X_train = []    
    img_names = train.image_name.values.tolist()
    for img_name in tqdm(img_names):
        img = get_img_cv2(train_img_path+img_name)
        X_train.append(img)
    X_train_id = train.row_id.values.tolist()
    X_train = np.array(X_train, dtype=np.int8)
    y_train = le.fit_transform(train.detected.values).astype(np.int8)
    
    return X_train, y_train, X_train_id

In [6]:
def load_test():
    X_test = []   
    img_names = test.image_name.values.tolist()
    for img_name in tqdm(img_names):
        img = get_img_cv2(test_img_path+img_name)
        X_test.append(img)
        
    X_test = np.array(X_test, dtype=np.int8)   
    test_id = test.row_id.values
    
    return X_test, test_id

In [7]:
def read_and_normalise_train():
    train_img, train_target, train_id = load_train()
    
    print('Reshape train_img for Tensorflow...')
    train_img = train_img.transpose((0, 1, 2, 3))
    
    print('convert to float and normalize....')
    train_img = train_img.astype(np.float32)
    train_img = train_img/255.
    train_target = to_categorical(train_target, 14) 
    
    print('Train shpae: {} Train target shape: {}'.format(train_img.shape, train_target.shape))
    
    return train_img, train_target, train_id

In [8]:
def read_and_normalise_test():
    test_img, test_id = load_test()
    
    print('Resahpe test_img for Tesnorflow...')
    test_img = test_img.transpose((0, 1, 2, 3))
    
    print('convert to float and normalize...')
    test_img = test_img.astype(np.float32)
    test_img = test_img/255.
    
    print('Test shpae: {}'.format(test_img.shape))
    return test_img, test_id
    

In [9]:
X_train, y_train, train_id = read_and_normalise_train()

100%|██████████| 18577/18577 [03:41<00:00, 83.82it/s]


Reshape train_img for Tensorflow...
convert to float and normalize....
Train shpae: (18577, 139, 139, 3) Train target shape: (18577, 14)


In [10]:
#Transfer learning with Inception V3 
#include_top=False excludes final FC layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IM_WIDTH, IM_HEIGHT, 3))  


In [11]:
## set model architechture 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
predictions = Dense(y_train.shape[1], activation='softmax')(x) #new softmax layer
model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [12]:
train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
)

In [13]:
train_generator = train_datagen.flow(
    X_train, y_train,
    #target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=BAT_SIZE,
)

In [14]:
history_tl = model.fit_generator(
    train_generator,
    epochs=NB_EPOCHS,
    steps_per_epoch=X_train.shape[0]/BAT_SIZE,     
    class_weight='auto')

Epoch 1/10
581/580 [==============================] - 70s - loss: 2.3491 - acc: 0.2845    
Epoch 2/10
581/580 [==============================] - 68s - loss: 2.1873 - acc: 0.3069    
Epoch 3/10
581/580 [==============================] - 67s - loss: 2.1718 - acc: 0.3079    
Epoch 4/10
581/580 [==============================] - 67s - loss: 2.1679 - acc: 0.3087    
Epoch 5/10
581/580 [==============================] - 67s - loss: 2.1575 - acc: 0.3130    
Epoch 6/10
581/580 [==============================] - 66s - loss: 2.1646 - acc: 0.3114    
Epoch 7/10
581/580 [==============================] - 66s - loss: 2.1598 - acc: 0.3112    
Epoch 8/10
581/580 [==============================] - 66s - loss: 2.1566 - acc: 0.3130    
Epoch 9/10
581/580 [==============================] - 65s - loss: 2.1520 - acc: 0.3152    
Epoch 10/10
581/580 [==============================] - 65s - loss: 2.1551 - acc: 0.3125    


In [15]:
for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
     layer.trainable = False
for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
     layer.trainable = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
history_ft = model.fit_generator(
    train_generator,
    epochs=NB_EPOCHS,
    steps_per_epoch=X_train.shape[0]// BAT_SIZE,    
    class_weight='auto')

Epoch 1/10
580/580 [==============================] - 128s - loss: 2.1341 - acc: 0.3120   
Epoch 2/10
580/580 [==============================] - 127s - loss: 2.1180 - acc: 0.3185   
Epoch 3/10
580/580 [==============================] - 127s - loss: 2.1131 - acc: 0.3169   
Epoch 4/10
580/580 [==============================] - 126s - loss: 2.0959 - acc: 0.3200   
Epoch 5/10
580/580 [==============================] - 124s - loss: 2.0898 - acc: 0.3256   
Epoch 6/10
580/580 [==============================] - 125s - loss: 2.0877 - acc: 0.3229   
Epoch 7/10
580/580 [==============================] - 125s - loss: 2.0821 - acc: 0.3273   
Epoch 8/10
580/580 [==============================] - 125s - loss: 2.0724 - acc: 0.3268   
Epoch 9/10
580/580 [==============================] - 125s - loss: 2.0741 - acc: 0.3262   
Epoch 10/10
580/580 [==============================] - 122s - loss: 2.0644 - acc: 0.3286   


In [18]:
X_test, test_id = read_and_normalise_test()

100%|██████████| 12386/12386 [02:33<00:00, 80.51it/s]


Resahpe test_img for Tesnorflow...
convert to float and normalize...
Test shpae: (12386, 139, 139, 3)


In [24]:
pred = model.predict(preprocess_input(X_test))


In [25]:
pred = le.inverse_transform(np.argmax(pred, axis=1))
sub = pd.DataFrame({'row_id':test_id, 'detected':pred})
sub = sub[['row_id', 'detected']]
sub.head(5)

,row_id,detected
0,id_100,class_3
1,id_10002,class_3
2,id_10005,class_5
3,id_10008,class_3
4,id_10009,class_3


In [26]:
sub.to_csv('incepv3_1.csv', index=False)

In [27]:
sub.detected.unique()

array(['class_3', 'class_5', 'class_7', 'class_12'], dtype=object)